ACTUALIZA LAS PROPIEDADES QUE TIENEN MIDIFICACIONES

In [1]:
import ee
ee.Authenticate()

Enter verification code:  4/1AVMBsJiq3TddRNcMjPGqy9pPdSt7JX-PzEztPEX-PrfLBIW_BPBsiAqkAV4



Successfully saved authorization token.


In [2]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [3]:
ee.Initialize()

In [4]:
ruta_catastro='projects/ee-bismarksr17/assets/CAMPODULCE'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_02_2025'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

In [5]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

## Eliminar propiedades sobra

In [7]:
# props que presentan irregularidades
lista_eliminar = [106, 622, 1080, 1172, 1302, 1413, 1480, 1540, 1626, 1986,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
    1998, 1999, 2001, 2002, 1421, 1962, 1289, 988, 986, 1446,
    1454, 637, 15, 636, 92, 1735, 880, 328, 19, 201, 114, 437,
    216, 1447, 365, 47, 123, 249, 132, 17, 1491, 1015, 85, 325,
    367, 491, 478, 1081, 744, 752, 313, 616, 1265, 1482, 1743]

In [7]:
df_intersects = gpd.read_file('INTERSECT_2.shp')

In [28]:
df_data = pd.read_excel('DATA_EST.xlsx')

### Eliminar props de DATA_EST.xlsx

In [29]:
df_data = df_data[~df_data['cod_prop'].isin(lista_eliminar)]

In [30]:
df_data.to_excel('data_new.xlsx')

### Eliminar props de INTERSECTS.shp

In [14]:
lista_eliminar = [str(i) for i in lista_eliminar]

In [15]:
df_intersects = df_intersects[~df_intersects['unidad_01'].isin(lista_eliminar)]

In [8]:
path_cat = r'G:\BISMARK\_TEMP\CAMPODULCE.shp'
gdf_cat = gpd.read_file(path_cat)

In [11]:
lista_props = list(set(gdf_cat['unidad_01']))
print(lista_props)

[480.0, 577.0, 482.0, 259.0, 299.0, 30.0]


In [10]:
lista_props = [2190.0, 2192.0, 2194.0, 2195.0, 2197.0, 2198.0, 2199.0, 2200.0, 2201.0, 2202.0, 2203.0, 2204.0, 2205.0, 2206.0, 2207.0, 2208.0, 2209.0, 2210.0, 2211.0, 2213.0, 2214.0, 2215.0, 2217.0, 2218.0, 2219.0, 2220.0, 2221.0, 2222.0, 2224.0, 2226.0, 2227.0, 2228.0, 2229.0, 2230.0, 2232.0, 2233.0, 2235.0, 2236.0, 2239.0, 2241.0, 2242.0, 2243.0, 2244.0, 2245.0, 2247.0, 2248.0, 2249.0, 2251.0, 2258.0, 2259.0, 2261.0, 2262.0, 2263.0, 2264.0, 2265.0, 2266.0, 2267.0, 2268.0, 2269.0, 2272.0, 2273.0, 2275.0, 2276.0, 2277.0, 2278.0, 2279.0, 2280.0, 2281.0, 2285.0, 2286.0, 2289.0, 2290.0, 2291.0, 2292.0, 2293.0, 2294.0]

In [12]:
cods_error

NameError: name 'cods_error' is not defined

In [13]:
len(lista_props)

6

In [14]:
df_intersects = gpd.read_file('INTERSECT_0.shp')

In [15]:
df_intersects.shape

(0, 27)

In [15]:
cods_error = []

In [16]:
contador = 0

for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_gdf(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    try:
        intersect = overlay(lotes_local, vector_local, how="intersection")
    except TypeError as e:
        cods_error.append(cod_prop)
        continue

    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
    
print('FIN......!')

inicio: 480.0


C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:63: FutureWarning: The provided callable <function sum at 0x000001D3E6D7E560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future vers

fin:  480.0
___________________CONTADOR:  1
inicio: 577.0


C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:63: FutureWarning: The provided callable <function sum at 0x000001D3E6D7E560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future vers

fin:  577.0
___________________CONTADOR:  2
inicio: 482.0


...

C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:63: FutureWarning: The provided callable <function sum at 0x000001D3E6D7E560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)


fin:  299.0
___________________CONTADOR:  5
inicio: 30.0


C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")


fin:  30.0
___________________CONTADOR:  6
FIN......!


C:\Users\Usuario\AppData\Local\Temp\ipykernel_35292\1897760324.py:63: FutureWarning: The provided callable <function sum at 0x000001D3E6D7E560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)


In [21]:
cods_error

NameError: name 'cods_error' is not defined

In [22]:
df_intersects.tail(5)

,area,cultivo,fecha,financia,id,soca,textura,unidad_01,unidad_02,unidad_03,...,geometry,codco,delegado,estado,fci,fid_1,incentivo,semana,tipoc,zafra
28239,26.881135,canha,NaN,PROPIO,30|41594|EP-L5,0,F,30,CAMPO DULCE,41594,...,"POLYGON ((501810.000 8096207.041, 501810.000 8...",41.0,AG CAMPO DULCE SRL,cosechado,4.0,20310.0,,2.0,sinmadurador,2025.0
28240,26.881135,canha,NaN,PROPIO,30|41594|EP-L5,0,F,30,CAMPO DULCE,41594,...,"POLYGON ((501817.789 8096167.093, 501813.811 8...",41.0,AG CAMPO DULCE SRL,cosechado,4.0,20310.0,,2.0,sinmadurador,2025.0
28241,26.881135,canha,NaN,PROPIO,30|41594|EP-L5,0,F,30,CAMPO DULCE,41594,...,"POLYGON ((501818.661 8096170.000, 501810.000 8...",41.0,AG CAMPO DULCE SRL,cosechado,4.0,20310.0,,2.0,sinmadurador,2025.0
28242,26.881135,canha,NaN,PROPIO,30|41594|EP-L5,0,F,30,CAMPO DULCE,41594,...,"POLYGON ((501820.000 8096190.000, 501820.000 8...",41.0,AG CAMPO DULCE SRL,cosechado,4.0,20310.0,,2.0,sinmadurador,2025.0
28243,26.881135,canha,NaN,PROPIO,30|41594|EP-L5,0,F,30,CAMPO DULCE,41594,...,"POLYGON ((501820.000 8096198.419, 501820.000 8...",41.0,AG CAMPO DULCE SRL,cosechado,4.0,20310.0,,2.0,sinmadurador,2025.0


In [24]:
len(df_intersects)

28244

In [25]:
df_intersects.to_file("INTERSECT_01.shp", driver="ESRI Shapefile")

In [23]:
df_intersects['area_2'].sum()

1781.5946083205763

In [27]:
df_intersects.head(3)

,area,cultivo,fecha,financia,id,soca,textura,unidad_01,unidad_02,unidad_03,...,geometry,codco,delegado,estado,fci,fid_1,incentivo,semana,tipoc,zafra
0,45.320456,canha,NaN,IAG,480|41594|Ca-1,4,FY,480,EL CANAL,41594,...,"POLYGON ((517825.213 8098291.622, 517830.000 8...",41.0,AG CAMPO DULCE SRL,sincosecha,0.0,19812.0,,0.0,sinmadurador,2025.0
1,45.320456,canha,NaN,IAG,480|41594|Ca-1,4,FY,480,EL CANAL,41594,...,"POLYGON ((517830.000 8098273.756, 517830.000 8...",41.0,AG CAMPO DULCE SRL,sincosecha,0.0,19812.0,,0.0,sinmadurador,2025.0
2,45.320456,canha,NaN,IAG,480|41594|Ca-1,4,FY,480,EL CANAL,41594,...,"POLYGON ((517840.000 8098290.000, 517840.000 8...",41.0,AG CAMPO DULCE SRL,sincosecha,0.0,19812.0,,0.0,sinmadurador,2025.0
